# Two-Stages Deep Learning Project

<p>This project has been developped by the Montreal Hacknight community. It derives from the braindecode framework and is aimed at being integrable into the Moabb project.</p>

<p><ul>https://github.com/robintibor/braindecode</ul>
<ul>https://github.com/NeuroTechX/moabb</ul></p>

<p>This project wouldn't have been possible without the contribution of these people: Arna (...), Justin (...), Yannick Roy, Eamon Egan, (...) and it is currently being supported by Fred Simard (fs@re-ak.com)</p>

<p>This project consist in deriving a two-stages training process from the brain decode implementation of a Shallow Convolutional Network. The core motivation is to combine the strenghts of deep learning - that is being one of the most powerful ML algorithms out-there - while grinding down it's fall back - it requires an immense of data.</p>

<p>The goal is to develop a method to train the network in two stages:</p>
<p><ul>1) Train the network over a large dataset, this training forms some kind of prior over the network</ul>
<ul>2) Train the network a second time, this time on the dataset of interest</ul>
</p>

<p>The premise motivating this approach is that the second training is likely to proceed faster than what is normally required to train a deep net, without sacrificing the performance; and, we suspect that the second training will be able to exploit the specificities of the dataset of interest in a way to surpasses the first training, which was trained on a generalistic dataset.</p>

In [0]:
"""
#
# Download the github, which contains the package with the networks
#
"""

# reset to default base path
import os
os.chdir('/content/')

# if dl-eeg-playground already here, erase
!rm -rf dl-eeg-playground

# clone dl-eeg-playground and cd to it
!git clone https://github.com/NeuroTechX/dl-eeg-playground.git
os.chdir('dl-eeg-playground/brainDecode/towardMoabbIntegration')


Cloning into 'dl-eeg-playground'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 286 (delta 13), reused 22 (delta 12), pack-reused 261
Receiving objects: 100% (286/286), 1.97 MiB | 2.15 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [0]:
#
# Download the whole BNCI 002-2014 dataset
#
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14E.mat

# move files into a dedicated folder
!mkdir BBCIData
!mv *.mat BBCIData

#install brain decode
!pip install braindecode -q

# install pytorch
# ref: http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
#  cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
import torch

import scipy.io as sio
import numpy as np
from os import listdir
from os.path import isfile, join

# prepare data containers
y = []
X = []

"""
Now, let's load data.

We read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
    - the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref: Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""

folder = "BBCIData"

for f in listdir(folder):
    # read file
    d1T = sio.loadmat(folder + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = 7*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print()
print(X.shape)
print(y.shape)


# rand permute dataset
idx = np.random.permutation(X.shape[0])

X = X[idx,:,:]
y = y[idx]

--2018-12-06 23:51:20--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-12-06 23:51:20--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  6.83MB/s    in 11s     

2018-12-06 23:51:33 (3.41 MB/s) - ‘S01T.mat’ saved [39794870/39794870]

--2018-12-06 23:51:34--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.

In [0]:

# load the general network Trainer
from brainDecodeSKLearnWrapper.ShallowFBCSPNet_GeneralTrainer import ShallowFBCSPNet_GeneralTrainer

classifier = ShallowFBCSPNet_GeneralTrainer(nb_epoch=60)
classifier.fit(X,y)


Epoch 0
Train  Loss: 2.08115
Train  Accuracy: 49.6%
Test   Loss: 2.04731
Test   Accuracy: 52.1%
Epoch 1
Train  Loss: 0.75969
Train  Accuracy: 50.4%
Test   Loss: 0.75388
Test   Accuracy: 53.2%
Epoch 2
Train  Loss: 1.25856
Train  Accuracy: 49.6%
Test   Loss: 1.23456
Test   Accuracy: 52.1%
Epoch 3
Train  Loss: 0.70035
Train  Accuracy: 56.2%
Test   Loss: 0.72591
Test   Accuracy: 53.9%
Epoch 4
Train  Loss: 0.69096
Train  Accuracy: 55.5%
Test   Loss: 0.72498
Test   Accuracy: 51.1%
Epoch 5
Train  Loss: 0.67922
Train  Accuracy: 56.9%
Test   Loss: 0.72395
Test   Accuracy: 52.5%
Epoch 6
Train  Loss: 0.68005
Train  Accuracy: 56.7%
Test   Loss: 0.72818
Test   Accuracy: 50.4%
Epoch 7
Train  Loss: 0.66926
Train  Accuracy: 58.4%
Test   Loss: 0.70863
Test   Accuracy: 52.9%
Epoch 8
Train  Loss: 0.65986
Train  Accuracy: 60.5%
Test   Loss: 0.69376
Test   Accuracy: 50.7%
Epoch 9
Train  Loss: 0.65886
Train  Accuracy: 60.8%
Test   Loss: 0.70577
Test   Accuracy: 50.7%
Epoch 10
Train  Loss: 0.73277
Train  Acc

ShallowFBCSPNet_GeneralTrainer(filter_time_length=75, n_filters_spat=5,
                n_filters_time=10, nb_epoch=60, pool_time_length=60,
                pool_time_stride=30)

In [0]:
classifier.model.parameters()

<generator object Module.parameters at 0x7fa40a6c1990>

In [0]:
from torch import optim
from torch import nn
from braindecode.datautil.signal_target import SignalAndTarget

# define a number of train/test trials
#nb_train_trials = int(np.floor(7/8*X.shape[0]))
classifier.model.conv_classifier = nn.Conv2d(5, 2,(116, 1), bias=True).cuda()

# split the dataset
#train_set = SignalAndTarget(X[:nb_train_trials], y=y[:nb_train_trials]) # confound, sets are mixed again
#test_set = SignalAndTarget(X[nb_train_trials:], y=y[nb_train_trials:])
train_set = classifier.train_set
test_set = classifier.test_set # that's the fix, but need to update classifier

classifier.optimizer = optim.Adam(classifier.model.parameters())

nbIter = 50

# array that tracks results
classifier.loss_rec = np.zeros((nbIter,2))
classifier.accuracy_rec = np.zeros((nbIter,2))

# run all epoch
for i_epoch in range(nbIter):

    classifier._batchTrain(i_epoch, train_set)
    classifier._evalTraining(i_epoch, train_set, test_set)

Epoch 0
Train  Loss: 0.45825
Train  Accuracy: 79.7%
Test   Loss: 0.48396
Test   Accuracy: 76.4%
Epoch 1
Train  Loss: 0.43202
Train  Accuracy: 78.1%
Test   Loss: 0.49042
Test   Accuracy: 75.0%
Epoch 2
Train  Loss: 0.42766
Train  Accuracy: 77.3%
Test   Loss: 0.50944
Test   Accuracy: 72.9%
Epoch 3
Train  Loss: 0.67299
Train  Accuracy: 64.4%
Test   Loss: 0.79049
Test   Accuracy: 57.9%
Epoch 4
Train  Loss: 0.34711
Train  Accuracy: 86.8%
Test   Loss: 0.43078
Test   Accuracy: 79.3%
Epoch 5
Train  Loss: 0.85764
Train  Accuracy: 59.7%
Test   Loss: 0.98398
Test   Accuracy: 55.0%
Epoch 6
Train  Loss: 0.73597
Train  Accuracy: 63.9%
Test   Loss: 0.85601
Test   Accuracy: 58.6%
Epoch 7
Train  Loss: 0.46463
Train  Accuracy: 75.3%
Test   Loss: 0.58487
Test   Accuracy: 71.1%
Epoch 8
Train  Loss: 0.33387
Train  Accuracy: 86.5%
Test   Loss: 0.45445
Test   Accuracy: 77.1%
Epoch 9
Train  Loss: 0.32303
Train  Accuracy: 87.7%
Test   Loss: 0.45208
Test   Accuracy: 78.9%
Epoch 10
Train  Loss: 0.31797
Train  Acc

In [0]:
from torch import optim
from torch import nn
from braindecode.datautil.signal_target import SignalAndTarget

!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat
!mkdir BBCIData
!mv *.mat BBCIData


"""
Now, let's load data.

We read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
    - the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref: Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""

folder = "BBCIData"
datasetID = 1

# prepare data containers
y_subset = []
X_subset = []

f = listdir(folder)[datasetID]
  
# read file
d1T = sio.loadmat(folder + "/" + f)

samplingRate = d1T['data'][0][0][0][0][3][0][0]
trialLength = 7*samplingRate

# run through all training runs
for run in range(len(d1T['data'][0])):
    y_subset.append(d1T['data'][0][run][0][0][2][0]) # labels
    timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
    rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

    # parse out data based on timestamps
    for start in timestamps:
        end = start + trialLength
        X_subset.append(rawData[:,start:end]) #15 x 2560

del rawData
del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X_subset = np.array(X_subset).astype(np.float32)
y_subset = (np.array(y_subset).flatten()-1).astype(np.int64)
print()
print(X_subset.shape)
print(y_subset.shape)


# rand permute dataset
idx = np.random.permutation(X_subset.shape[0])

X_subset = X_subset[idx,:,:]
y_subset = y_subset[idx]


# define a number of train/test trials
nb_train_trials = int(np.floor(7/8*X_subset.shape[0]))

# split the dataset
train_set_subset = SignalAndTarget(X_subset[:nb_train_trials], y=y_subset[:nb_train_trials])
test_set_subset = SignalAndTarget(X_subset[nb_train_trials:], y=y_subset[nb_train_trials:])







from torch import optim
from torch import nn
from braindecode.datautil.signal_target import SignalAndTarget

# define a number of train/test trials
classifier.model.conv_classifier = nn.Conv2d(5, 2,(116, 1), bias=True).cuda()

classifier.optimizer = optim.Adam(classifier.model.parameters())

nbIter = 50

# array that tracks results
classifier.loss_rec = np.zeros((nbIter,2))
classifier.accuracy_rec = np.zeros((nbIter,2))

# run all epoch
for i_epoch in range(nbIter):

    classifier._batchTrain(i_epoch, train_set_subset)
    classifier._evalTraining(i_epoch, train_set_subset, test_set_subset)


--2018-12-07 00:14:15--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-12-07 00:14:15--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  6.84MB/s    in 12s     

2018-12-07 00:14:28 (3.27 MB/s) - ‘S01T.mat’ saved [39794870/39794870]

--2018-12-07 00:14:29--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.

In [0]:
nbIter = 50

# array that tracks results
classifier.loss_rec = np.zeros((nbIter,2))
classifier.accuracy_rec = np.zeros((nbIter,2))

# run all epoch
for i_epoch in range(nbIter):

    classifier._batchTrain(i_epoch, train_set_subset)
    classifier._evalTraining(i_epoch, train_set_subset, test_set_subset)


Epoch 0
Train  Loss: 0.01411
Train  Accuracy: 100.0%
Test   Loss: 0.45631
Test   Accuracy: 87.5%
Epoch 1
Train  Loss: 0.01369
Train  Accuracy: 100.0%
Test   Loss: 0.45678
Test   Accuracy: 87.5%
Epoch 2
Train  Loss: 0.01326
Train  Accuracy: 100.0%
Test   Loss: 0.45545
Test   Accuracy: 87.5%
Epoch 3
Train  Loss: 0.01289
Train  Accuracy: 100.0%
Test   Loss: 0.45607
Test   Accuracy: 87.5%
Epoch 4
Train  Loss: 0.01253
Train  Accuracy: 100.0%
Test   Loss: 0.45559
Test   Accuracy: 87.5%
Epoch 5
Train  Loss: 0.01216
Train  Accuracy: 100.0%
Test   Loss: 0.45258
Test   Accuracy: 87.5%
Epoch 6
Train  Loss: 0.01182
Train  Accuracy: 100.0%
Test   Loss: 0.44866
Test   Accuracy: 87.5%
Epoch 7
Train  Loss: 0.01151
Train  Accuracy: 100.0%
Test   Loss: 0.44688
Test   Accuracy: 87.5%
Epoch 8
Train  Loss: 0.01122
Train  Accuracy: 100.0%
Test   Loss: 0.44529
Test   Accuracy: 87.5%
Epoch 9
Train  Loss: 0.01096
Train  Accuracy: 100.0%
Test   Loss: 0.44381
Test   Accuracy: 87.5%
Epoch 10
Train  Loss: 0.01070


In [0]:
"""
#from brainDecodeSKLearnWrapper.ShallowFBCSPNet_SpecializedTrainer import ShallowFBCSPNet_SpecializedTrainer


!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
!mkdir BBCIData
!mv *.mat BBCIData


"""
Now, let's load data.

We read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
    - the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref: Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""

folder = "BBCIData"
datasetID = 1

# prepare data containers
y = []
X = []

f = listdir(folder)[datasetID]
  
# read file
d1T = sio.loadmat(folder + "/" + f)

samplingRate = d1T['data'][0][0][0][0][3][0][0]
trialLength = 7*samplingRate

# run through all training runs
for run in range(len(d1T['data'][0])):
    y.append(d1T['data'][0][run][0][0][2][0]) # labels
    timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
    rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

    # parse out data based on timestamps
    for start in timestamps:
        end = start + trialLength
        X.append(rawData[:,start:end]) #15 x 2560

del rawData
del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print()
print(X.shape)
print(y.shape)


# rand permute dataset
idx = np.random.permutation(X.shape[0])

X = X[idx,:,:]
y = y[idx]







#specializedClassifier = ShallowFBCSPNet_SpecializedTrainer(network=classifier.model)
#specializedClassifier.lr = 0.01
#specializedClassifier.configure(initial_lr=0.01)
#specializedClassifier.fit(X,y)


"""



--2018-10-12 01:02:30--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-10-12 01:02:30--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  10.4MB/s    in 5.9s    

2018-10-12 01:02:37 (6.44 MB/s) - ‘S01T.mat’ saved [39794870/39794870]

--2018-10-12 01:02:38--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.

In [0]:



specializedClassifier = ShallowFBCSPNet_SpecializedTrainer(network=classifier.model)
specializedClassifier.lr = 0.01
#specializedClassifier.configure(initial_lr=0.01)
specializedClassifier.fit(X,y)


Epoch 0
Train  Loss: 0.40867
Train  Accuracy: 79.3%
Test   Loss: 1.06219
Test   Accuracy: 38.5%
Epoch 1
Train  Loss: 0.24243
Train  Accuracy: 97.7%
Test   Loss: 1.22067
Test   Accuracy: 30.8%
Epoch 2
Train  Loss: 0.15374
Train  Accuracy: 98.9%
Test   Loss: 1.23056
Test   Accuracy: 46.2%
Epoch 3
Train  Loss: 0.10635
Train  Accuracy: 100.0%
Test   Loss: 1.24592
Test   Accuracy: 53.8%
Epoch 4
Train  Loss: 0.07798
Train  Accuracy: 100.0%
Test   Loss: 1.24732
Test   Accuracy: 46.2%
Epoch 5
Train  Loss: 0.05839
Train  Accuracy: 100.0%
Test   Loss: 1.29423
Test   Accuracy: 46.2%
Epoch 6
Train  Loss: 0.04658
Train  Accuracy: 100.0%
Test   Loss: 1.35518
Test   Accuracy: 46.2%
Epoch 7
Train  Loss: 0.03919
Train  Accuracy: 100.0%
Test   Loss: 1.43166
Test   Accuracy: 46.2%
Epoch 8
Train  Loss: 0.03243
Train  Accuracy: 100.0%
Test   Loss: 1.48398
Test   Accuracy: 46.2%
Epoch 9
Train  Loss: 0.02648
Train  Accuracy: 100.0%
Test   Loss: 1.53610
Test   Accuracy: 46.2%
Epoch 10
Train  Loss: 0.02174
Tra

ShallowFBCSPNet_SpecializedTrainer(filename=None, network=None)

In [0]:


Subject 1:
Train  Loss: 0.01462
Train  Accuracy: 100.0%
Test   Loss: 0.46110
Test   Accuracy: 87.5%

  
Subject 2:
Train  Loss: 0.02354
Train  Accuracy: 100.0%
Test   Loss: 0.22113
Test   Accuracy: 100.0%  


Best scores so far:

Subject 0: 76.9%
Subject 1 50.0%


